In [450]:
import cv2
# from flask import Flask 
import numpy as np
import os
import math
import plotly.graph_objects as go
import time
from PIL import Image
from level_3 import mask_img,leavel

In [452]:
t_prev = time.time()
h_min = 50
s_min = 0
v_min = 0
h_max = 179
s_max = 179
v_max = 110

In [453]:
kelvin_table = {
    1000: (255,56,0),
    1500: (255,109,0),
    2000: (255,137,18),
    2500: (255,161,72),
    3000: (255,180,107),
    3500: (255,196,137),
    4000: (255,209,163),
    4500: (255,219,186),
    5000: (255,228,206),
    5500: (255,236,224),
    6000: (255,243,239),
    6500: (255,249,253),
    7000: (245,243,255),
    7500: (235,238,255),
    8000: (227,233,255),
    8500: (220,229,255),
    9000: (214,225,255),
    9500: (208,222,255),
    10000: (204,219,255)}

In [454]:
all_files = []
all_paths = []
leavel_list = []
path_1="G:\\Harden_project\\angle_test_all_0304\\9.17.00\\120_yolo\\" # 資料夾
# dir_1 = "91700120"   # 資料夾
# people_leveal = 3  # 人判等級
# eg = 19       # 題目 
# ang = '120'     # 角度

In [455]:
def convert_temp(image, temp):
    r, g, b = kelvin_table[temp]
    matrix = ( r / 255.0, 0.0, 0.0, 0.0,
               0.0, g / 255.0, 0.0, 0.0,
               0.0, 0.0, b / 255.0, 0.0 )
    return image.convert('RGB', matrix)

In [456]:
def tem(img):   
    img2= Image.fromarray(np.uint8(img))
    w = convert_temp(img2,6500)
    img = np.asarray(w)
    imgHSV = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    lower=np.array([h_min,s_min,v_min])
    upper=np.array([h_max,s_max,v_max])
    mask=cv2.inRange(imgHSV,lower,upper)
    #cv2.imshow("mask",mask)
    #cv2.imshow("img",img)
    #cv2.waitKey()

    return mask,imgHSV

In [457]:
#DataPath
def datapath_dir(all_files,all_paths,path_1):
    for path, dir, file in os.walk(path_1): #資料夾路徑
        for f in file:
            if os.path.splitext(f)[-1] in ['.JPG']:
                all_files.append(f)
                all_paths.append(path)
    return all_files,all_paths

all_files,all_paths= datapath_dir(all_files=all_files,all_paths=all_paths,path_1=path_1)

In [458]:
def leavel(black,ori):
    black_h,black_w = black.shape[0] ,  black.shape[1]
    level_s = [[255,255,255],[209,210,212],[167,169,172],[129,130,133],[88,88,90],[35,31,32]] ## 林格曼標籤分級
    # level_s = [[0,0,255],[110,3,212],[108,7,172],[112,7,133],[120,5,90],[172,29,35]]
    res = [0] * 6 

    for i in range(black_h): ##訪問每個像素
        for j in range(black_w):
            if black[i][j] >= 0 :  ###  背景
                level_s2,d = 0 , 1000000000000000000000             ##不是背景  判斷哪一級
                for z in range(len(level_s)):
                    std_d = (abs( ori[i][j][0] -level_s[z][0] )**2+abs( ori[i][j][1] -level_s[z][1] )**2+abs( ori[i][j][2] -level_s[z][2] )**2)**0.5
                    # std_d = (abs( ori[i][j][2] -level_s[z][2] )**2)**0.5
                    if  std_d < d:
                            d,level_s2 = std_d,z
                res[level_s2] = res[level_s2] + 1
                #print("std_d",std_d)
                #print(res)
    #print('此照片不透視率 %s 級 有 %s 個  , 比例為 %s ' %   ( res.index(max(res)) , max(res) , round(max(res) / sum(res),2) * 100 ) )
    return res.index(max(res))

In [459]:
for k in range (len(all_files)):
    img = cv2.imread(all_paths[k]+ all_files[k])
    img = cv2.resize(img,(400,400))
    print(all_files[k],"  第 {} 張照片".format(k))
    mask,imgHSV = tem(img)

    try:
        ori,black  = mask_img(mask,img)
        #cv2.imshow("mask",black)
        #cv2.imshow("ori",ori)
        #cv2.waitKey()
        leavel_list.append(leavel(black,imgHSV))
        print("判斷黑煙等級: {} ".format(leavel(black,imgHSV)))
    except:
        pass

max_h= 399 mix_h= 320
max_w= 301 mix_w= 148
判斷黑煙等級: 1 
max_h= 399 mix_h= 334
max_w= 314 mix_w= 176
判斷黑煙等級: 1 
max_h= 399 mix_h= 327
max_w= 375 mix_w= 162
判斷黑煙等級: 1 
max_h= 399 mix_h= 327
max_w= 318 mix_w= 173
判斷黑煙等級: 1 
max_h= 399 mix_h= 330
max_w= 306 mix_w= 164
判斷黑煙等級: 1 
max_h= 399 mix_h= 324
max_w= 287 mix_w= 148
判斷黑煙等級: 1 
max_h= 399 mix_h= 327
max_w= 384 mix_w= 135
判斷黑煙等級: 1 
max_h= 399 mix_h= 315
max_w= 335 mix_w= 101
判斷黑煙等級: 1 
max_h= 399 mix_h= 322
max_w= 399 mix_w= 180
判斷黑煙等級: 1 


In [460]:
counts = np.bincount(leavel_list)
level_count = np.argmax(counts)
level_mean = np.mean(leavel_list)
level_var = np.var(leavel_list,ddof=1)
level_std = np.std(leavel_list,ddof=1)
print("mean=",level_mean,"variance=",level_var)

mean= 1.0 variance= 0.0


In [461]:
leavel_list

[1, 1, 1, 1, 1, 1, 1, 1, 1]

In [462]:
print("判定結果:" , int(level_count))

判定結果: 1


In [463]:
def normal_distribution(x, mean, var):
    return np.exp(-1*(abs(x-mean)**2)/(2*(var)))/(math.sqrt(2*np.pi) * (level_std))

x1 = np.linspace(level_mean - 6*level_std, level_mean + 6*level_std, 100)
ll=np.array(leavel_list)
x2 = np.linspace(1,6,6)
y2 = [np.count_nonzero(ll==0),np.count_nonzero(ll==1),np.count_nonzero(ll==2),np.count_nonzero(ll==3),np.count_nonzero(ll==4),np.count_nonzero(ll==5)]
y2=np.array(y2)